In [ ]:
%env JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'
%env FLINK_CONF_DIR=/Users/soufiane/Workspace/agile/medical-dashboard/back-end-synthea-flink/config

In [ ]:
%load_ext streaming_jupyter_integrations.magics

In [ ]:
%flink_connect --execution-target remote --remote-hostname localhost --remote-port 8081
#%flink_connect 

In [ ]:
%%flink_execute_sql
CREATE TABLE patient (
  `id` STRING,
  `gender` STRING,
  `name` ROW(family STRING ) ARRAY
) WITH (
  'connector' = 'kafka',
  'topic' = 'Patient',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

In [ ]:
%%flink_execute_sql
CREATE TABLE diagnosticReport (
  `id` STRING,
  `subject` ROW (`reference` STRING),
  `code` ROW(`coding` ROW(`display` STRING) ARRAY)
) WITH (
  'connector' = 'kafka',
  'topic' = 'DiagnosticReport',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

In [ ]:
%%flink_execute_sql
CREATE OR REPLACE TABLE hospital (
  `id` STRING,
  `identifier` ROW (`value` STRING) ARRAY,
  `status` STRING,
  `name` STRING,
  `telecom` ROW (`system` STRING, `value` STRING) ARRAY,
  `address` ROW (
    `line` STRING ARRAY,
    `city` STRING,
    `state` STRING,
    `postalCode` STRING,
    `country` STRING
  ),
  `position` ROW (
    `longitude` DOUBLE,
    `latitude` DOUBLE
  ),
  `managingOrganization` ROW (
    `identifier` ROW (`value` STRING)
  )
) WITH (
  'connector' = 'kafka',
  'topic' = 'Location',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

In [ ]:
%%flink_execute_sql
SHOW TABLES

In [ ]:
%%flink_execute_sql
SELECT * FROM patient

In [ ]:
%%flink_execute_sql
SELECT * FROM diagnosticReport

In [ ]:
%%flink_execute_sql
SELECT * FROM hospital

In [ ]:
%%flink_execute_sql
SELECT id, name[1].family as familyName, gender FROM patient limit 5

In [ ]:
%%flink_execute_sql
SELECT id,subject.reference, code.coding[1].display as name FROM diagnosticReport limit 5

In [ ]:
%%flink_execute_sql
SELECT p.name[1].family as familyName, p.gender, dr.id, dr.code.coding[1].display as diagnosticReportLabel
FROM diagnosticReport dr
JOIN patient p ON CONCAT('urn:uuid:', p.id) = dr.subject.reference

## Insert into Postgres

Create table in Postgres (replace container name by yours)

```bash
docker exec -it flink-sql_tutorial-db-1 psql -U example
> create table patient(identifier VARCHAR, familyName VARCHAR, gender VARCHAR, diagnosticReportLabel VARCHAR, PRIMARY KEY(identifier));
```

In [ ]:
%%flink_execute_sql
CREATE TABLE patient_psql (
  `identifier` STRING,
  `familyName` STRING,
  `gender` STRING,
  `diagnosticReportLabel` STRING,
  PRIMARY KEY (`identifier`) NOT ENFORCED
) WITH (
  'connector' = 'jdbc',
  'url' = 'jdbc:postgresql://db:5432/example',
  'table-name' = 'public.patient',
  'username' = 'example',
  'password' = 'example'  
)

In [ ]:
%%flink_execute_sql
INSERT INTO patient_psql
SELECT dr.id, p.name[1].family as familyName, p.gender, dr.code.coding[1].display as diagnosticReportLabel
FROM diagnosticReport dr
JOIN patient p ON CONCAT('urn:uuid:', p.id) = dr.subject.reference